# Pengujian

Memastikan posisi eksekusi folder program di folder darknet

In [ ]:
%cd /content/darknet

Import library

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
import random
import darknet
from pprint import pprint

Memeriksa jumlah gambar yang dideteksi

In [ ]:
def check_batch_shape(images, batch_size):
   """
       Image sizes should be the same width and height
   """
   shapes = [image.shape for image in images]
   if len(set(shapes)) > 1:
       raise ValueError("Images don't have same shape")
   if len(shapes) > batch_size:
       raise ValueError("Batch size higher than number of images")
   return shapes[0]

Mengubah data gambar ke format yang dapat diproses model

In [ ]:
def prepare_batch(images, network, channels=3):
   width = darknet.network_width(network)
   height = darknet.network_height(network)
 
   darknet_images = []
   for image in images:
       image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
       image_resized = cv2.resize(image_rgb, (width, height),
                                  interpolation=cv2.INTER_LINEAR)
       custom_image = image_resized.transpose(2, 0, 1)
       darknet_images.append(custom_image)
 
   batch_array = np.concatenate(darknet_images, axis=0)
   batch_array = np.ascontiguousarray(batch_array.flat, dtype=np.float32)/255.0
   darknet_images = batch_array.ctypes.data_as(darknet.POINTER(darknet.c_float))
   return darknet.IMAGE(width, height, channels, darknet_images)

Memeriksa ukuran dan jumlah gambar kemudian mengubah ke format yang dapat diproses

In [ ]:
def batch_detection(network, images, class_names, class_colors,
                   thresh=0.25, hier_thresh=.5, nms=.45, batch_size=4):
   image_height, image_width, _ = check_batch_shape(images, batch_size)
   darknet_images = prepare_batch(images, network)
   batch_detections = darknet.network_predict_batch(network, darknet_images,
                                                    batch_size, image_width,
                                                    image_height, thresh,
                                                    hier_thresh, None, 0, 0)
   batch_predictions = []
   for idx in range(batch_size):
       num = batch_detections[idx].num
       detections = batch_detections[idx].dets
       if nms:
           darknet.do_nms_obj(detections, num, len(class_names), nms)
       predictions = darknet.remove_negatives(detections, class_names, num)
       images[idx] = darknet.draw_boxes(predictions, images[idx], class_colors)
       batch_predictions.append(predictions)
   darknet.free_batch_detections(batch_detections, batch_size)
 
   return images, batch_predictions 

Loading network

In [ ]:
batch_size = 16
network, class_names, class_colors = darknet.load_network(
       'agriculture/yolov4-tiny-agriculture.cfg',
       'agriculture/obj.data',
       'agriculture/weights/yolov4-tiny-agriculture_final.weights',
       batch_size=batch_size
)